<a href="https://colab.research.google.com/github/jyonalee/Insider-Threat-and-Anomaly-Detection-from-User-Activities/blob/master/Anomaly_Detection_LSTM_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install awscli to download the data
!pip3 install awscli --upgrade --user

# download data and save it on `data`
!mkdir data
!~/.local/bin/aws s3 sync --no-sign-request --region us-west-1 "s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/" data/.

# Anomaly Detection with LSTM in Network Traffic Data

This project explores anomaly detection in network traffic with RNN-LSTM to train the model.

The dataset can be obtained [here](https://www.unb.ca/cic/datasets/ids-2018.html)

This is part of the capstone project for the Machine Learning Nano Degree from Udacity

##Data Exploration

---



In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
# load data and do preliminary cleaning
directory = '/home/jlee/cse-cic-ids2018/Processed Traffic Data for ML Algorithms'

filepath = os.path.join(directory,'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv')
df1 = pd.read_csv(filepath)
df1 = df1[df1['Protocol'] != 'Protocol']
filepath = os.path.join(directory,'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv')
df2 = pd.read_csv(filepath)
df2 = df2[df2['Protocol'] != 'Protocol']
filepath = os.path.join(directory,'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv')
df3 = pd.read_csv(filepath)
filepath = os.path.join(directory,'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv')
df4 = pd.read_csv(filepath)
filepath = os.path.join(directory,'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv')
df5 = pd.read_csv(filepath)
filepath = os.path.join(directory,'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv')
df6 = pd.read_csv(filepath)
filepath = os.path.join(directory,'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv')
df7 = pd.read_csv(filepath)
filepath = os.path.join(directory,'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv')
df8 = pd.read_csv(filepath)
df8 = df8[df8['Protocol'] != 'Protocol']
filepath = os.path.join(directory,'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv')
df9 = pd.read_csv(filepath)


/home/jlee/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jlee/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jlee/miniconda3/lib/python3.6/site-packages

In [3]:
# filepath = os.path.join(directory,'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv')
# df10 = pd.read_csv(filepath)
# df10 = df10[df10['Protocol'] != 'Protocol']

In [4]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9], ignore_index=True)

In [5]:
# clean up intermediary dataframes to free memory
del df1
del df2
del df3
del df4
del df5
del df6
del df7
del df8
del df9


In [6]:
len(df)

8284195

In [7]:
# get count of each label
print(df['Label'].value_counts())

Benign                      6112151
DDOS attack-HOIC             686012
DoS attacks-Hulk             461912
Bot                          286191
FTP-BruteForce               193360
SSH-Bruteforce               187589
Infilteration                161934
DoS attacks-SlowHTTPTest     139890
DoS attacks-GoldenEye         41508
DoS attacks-Slowloris         10990
DDOS attack-LOIC-UDP           1730
Brute Force -Web                611
Brute Force -XSS                230
SQL Injection                    87
Name: Label, dtype: int64


In [8]:
# get distribution in of each label
print(df['Label'].value_counts()/len(df))

Benign                      0.737809
DDOS attack-HOIC            0.082810
DoS attacks-Hulk            0.055758
Bot                         0.034547
FTP-BruteForce              0.023341
SSH-Bruteforce              0.022644
Infilteration               0.019547
DoS attacks-SlowHTTPTest    0.016886
DoS attacks-GoldenEye       0.005011
DoS attacks-Slowloris       0.001327
DDOS attack-LOIC-UDP        0.000209
Brute Force -Web            0.000074
Brute Force -XSS            0.000028
SQL Injection               0.000011
Name: Label, dtype: float64


so in essence, 73.8% of data points in this dataset is 'Benign' while the rest are some form of malicious attack

In [14]:
# todo:
# more stats
# plot timeline of events
# determine what is a normal sequence vs not normal sequence and visualize if possible

In [15]:
# todo:
# with the definition of a `normal sequence`, need to transform & process the data accordingly before training with LSTM
# with the baseline algorithms for anomaly detection, should be fine to train with given data as-is